<a href="https://colab.research.google.com/github/pko89403/Recsys_test/blob/master/item2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import random
from collections import Counter
import pandas as pd

In [0]:
class SGNS(nn.Module): # Skip-Gram Model with Negative Sampling
  def __init__(self, total_items, embedding_dimensions):
    super(SGNS, self).__init__()
  
  def forward(self, pos_v, neg_v):
    pass

In [0]:
!pwd

/content


In [0]:
!ls /content/

sample_data


In [0]:
import pandas as pd

RATINGS_PATH = "/content/drive/My Drive/data/ml-1m/ratings.dat"
MOVIES_PATH = "/content/drive/My Drive/data/ml-1m/movies.dat"

In [0]:
ratings = pd.read_csv(RATINGS_PATH, header=None, sep="::", engine="python")
movies = pd.read_csv(MOVIES_PATH, header=None, sep="::", engine="python")

In [0]:
ratings.columns = ["uuid", "iid", "rating", "timestamp"]
movies.columns = ["iid", "movie_name", "genre"]

In [30]:
ratings.head()
movies.head()

,iid,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
iid_to_movie_name = dict( zip(movies.iid.tolist(), movies.movie_name.tolist()))
movie_name_to_iid = { v: k for k, v in iid_to_movie_name.items()}

In [0]:
iid_to_movie_name

In [0]:
movie_name_to_iid

Total Items는 중요하다. Total Items가 Word2Vec에서의 Vocab Size와 같다.

In [53]:
only_user = ratings.uuid.unique().tolist()
total_items = len(movie_name_to_iid)
print(f"Total Users : {len(only_user)}, Total Items : {total_items}")


Total Users : 6040, Total Items : 3883


유저 하나당 평점 매긴 영화들의 리스트를 만든다.    
왜냐면 그걸로 Item2Vec의 데이터로 사용할 것이기 때문에

In [0]:
skip_item_dataset = [ratings[ratings.uuid==user]['iid'].tolist() for user in only_user]

[1193, 661, 914, 3408, 2355, 1197, 1287, 2804, 594, 919, 595, 938, 2398, 2918, 1035, 2791, 2687, 2018, 3105, 2797, 2321, 720, 1270, 527, 2340, 48, 1097, 1721, 1545, 745, 2294, 3186, 1566, 588, 1907, 783, 1836, 1022, 2762, 150, 1, 1961, 1962, 2692, 260, 1028, 1029, 1207, 2028, 531, 3114, 608, 1246]
